# Multiclass Classification Neural Network

In [1]:
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

In [11]:
df_train = pd.read_csv('datasets/customer_seg_train.csv', index_col='ID')
df_train.dropna(inplace=True)
df_train.head()

,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
ID,,,,,,,,,,
462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,D
466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,B
461735,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6,B
461319,Male,Yes,56,No,Artist,0.0,Average,2.0,Cat_6,C
460156,Male,No,32,Yes,Healthcare,1.0,Low,3.0,Cat_6,C


In [12]:
df_train['Ever_Married'] = df_train['Ever_Married'].map({'Yes': 1, 'No': 0})
df_train['Graduated'] = df_train['Graduated'].map({'Yes': 1, 'No': 0})
ohe_columns = pd.get_dummies(df_train[['Gender', 'Profession', 'Spending_Score']], prefix=['Gender', 'Profession', 'Spending_Score'], dtype='float')
df_train.drop(columns=['Gender', 'Profession', 'Spending_Score', 'Segmentation'], inplace=True)
df_train = pd.concat([df_train, ohe_columns], axis='columns')
df_train.head()

,Ever_Married,Age,Graduated,Work_Experience,Family_Size,Var_1,Gender_Female,Gender_Male,Profession_Artist,Profession_Doctor,Profession_Engineer,Profession_Entertainment,Profession_Executive,Profession_Healthcare,Profession_Homemaker,Profession_Lawyer,Profession_Marketing,Spending_Score_Average,Spending_Score_High,Spending_Score_Low
ID,,,,,,,,,,,,,,,,,,,,
462809,0,22,0,1.0,4.0,Cat_4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
466315,1,67,1,1.0,1.0,Cat_6,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
461735,1,67,1,0.0,2.0,Cat_6,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
461319,1,56,0,0.0,2.0,Cat_6,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
460156,0,32,1,1.0,3.0,Cat_6,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0


In [13]:
X_train = df_train.drop(columns=['Var_1'])
X_train

,Ever_Married,Age,Graduated,Work_Experience,Family_Size,Gender_Female,Gender_Male,Profession_Artist,Profession_Doctor,Profession_Engineer,Profession_Entertainment,Profession_Executive,Profession_Healthcare,Profession_Homemaker,Profession_Lawyer,Profession_Marketing,Spending_Score_Average,Spending_Score_High,Spending_Score_Low
ID,,,,,,,,,,,,,,,,,,,
462809,0,22,0,1.0,4.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
466315,1,67,1,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
461735,1,67,1,0.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
461319,1,56,0,0.0,2.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
460156,0,32,1,1.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
463002,1,41,1,0.0,5.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
464685,0,35,0,3.0,4.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
465406,0,33,1,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0


In [14]:
df_test = pd.read_csv('datasets/customer_seg_test.csv', index_col='ID')
df_test.dropna(inplace=True)
df_test['Ever_Married'] = df_test['Ever_Married'].map({'Yes': 1, 'No': 0})
df_test['Graduated'] = df_test['Graduated'].map({'Yes': 1, 'No': 0})
ohe_columns = pd.get_dummies(df_test[['Gender', 'Profession', 'Spending_Score']], prefix=['Gender', 'Profession', 'Spending_Score'], dtype='float')
df_test.drop(columns=['Gender', 'Profession', 'Spending_Score'], inplace=True)
df_test = pd.concat([df_test, ohe_columns], axis='columns')
X_test = df_test.drop(columns=['Var_1'])

In [15]:
X_train.shape, X_test.shape

((6665, 19), (2154, 19))

In [16]:
mm  = MinMaxScaler()
mm.fit(X_train)
X_train = mm.transform(X_train)
X_test = mm.transform(X_test)

### Classifcation using CategoricalCrossEntropy
Using this loss function, the target variables are one hot encoded

In [17]:
target_train_ohe = pd.get_dummies(df_train[['Var_1']], dtype='float')
target_train_ohe.head()

,Var_1_Cat_1,Var_1_Cat_2,Var_1_Cat_3,Var_1_Cat_4,Var_1_Cat_5,Var_1_Cat_6,Var_1_Cat_7
ID,,,,,,,
462809,0.0,0.0,0.0,1.0,0.0,0.0,0.0
466315,0.0,0.0,0.0,0.0,0.0,1.0,0.0
461735,0.0,0.0,0.0,0.0,0.0,1.0,0.0
461319,0.0,0.0,0.0,0.0,0.0,1.0,0.0
460156,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [18]:
y_train = target_train_ohe.values
y_test = pd.get_dummies(df_test[['Var_1']], dtype='float').values
y_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [20]:
total_classes = y_test.shape[1]
model = tf.keras.Sequential([
    tf.keras.Input(shape= (X_train.shape[1],) ),
    tf.keras.layers.Dense(units=12, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.01)),
    tf.keras.layers.Dense(units=total_classes, kernel_regularizer=tf.keras.regularizers.L2(0.01))
])
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                optimizer=tf.keras.optimizers.SGD(learning_rate=1.), metrics=['accuracy'])
model.fit(
    X_train,
    y_train,
    batch_size=10000,
    epochs=200)

Epoch 1/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.3050 - loss: 2.0376
Epoch 2/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 0.6716 - loss: 1.4907
Epoch 3/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - accuracy: 0.6716 - loss: 1.4180
Epoch 4/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step - accuracy: 0.6716 - loss: 1.3708
Epoch 5/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - accuracy: 0.6716 - loss: 1.3431
Epoch 6/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - accuracy: 0.6716 - loss: 1.3227
Epoch 7/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.6716 - loss: 1.3079
Epoch 8/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.6716 - loss: 1.2955
Epoch 9/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 162ms/step - accuracy: 0.6716 - loss: 1.2852
Epoch 10/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 0.6716 - loss: 1.2759
Epoch 11/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step - accuracy: 0.6716 - loss: 1.2675
Epoch 12/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - accuracy: 0.6716 - lo

In [21]:
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print('Loss (Mean error per testing example):', loss)
print('Accuracy: ', accuracy)

Loss (Mean error per testing example): 1.1032861471176147
Accuracy:  0.6597028970718384


### Classification using SparseCategoricalCrossEntropy
Using this loss function, target variables are encoded with integers.

In [22]:
le = LabelEncoder()
y_train2 = le.fit_transform(df_train[['Var_1']])
y_test2 = le.fit_transform(df_test[['Var_1']])
y_train2

c:\Users\chaud\anaconda3\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\chaud\anaconda3\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([3, 5, 5, ..., 5, 5, 3])

In [24]:
model2 = tf.keras.Sequential([
    tf.keras.Input(shape= (X_train.shape[1],) ),
    tf.keras.layers.Dense(units=12, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.01)),
    tf.keras.layers.Dense(units=total_classes, kernel_regularizer=tf.keras.regularizers.L2(0.01))
])
model2.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                optimizer=tf.keras.optimizers.SGD(learning_rate=1.), metrics=['accuracy'])
model2.fit(
    X_train,
    y_train2,
    batch_size=10000,
    epochs=100)

Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.0557 - loss: 2.4951
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.6716 - loss: 1.7064
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - accuracy: 0.6716 - loss: 1.4938
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 169ms/step - accuracy: 0.6716 - loss: 1.4196
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - accuracy: 0.6716 - loss: 1.3855
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - accuracy: 0.6716 - loss: 1.3595
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step - accuracy: 0.6716 - loss: 1.3391
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.6716 - loss: 1.3223
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step - accuracy: 0.6716 - loss: 1.3082
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - accuracy: 0.6716 - loss: 1.2960
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - accuracy: 0.6716 - loss: 1.2852
Epoch 12/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 168ms/step - accuracy: 0.6716 - l

In [25]:
loss, accuracy = model2.evaluate(X_test, y_test2, verbose=0)
print('Loss (Mean error per testing example):', loss)
print('Accuracy: ', accuracy)

Loss (Mean error per testing example): 1.1117568016052246
Accuracy:  0.6597028970718384


In [27]:
logit = model.predict(X_test, verbose=0) # This is the raw answer of equation wx+b
print('logit is:\n', logit, '\n')
# To convert this to probability generated by sigmoid function, use this command:
y_pred = np.array(tf.nn.softmax(logit))
print('y_pred is:\n', y_pred, '\n')

logit is:
 [[-1.1776707  -0.08133902  0.49458405 ... -1.4841962   2.2040477
  -0.80830616]
 [-1.2053437  -0.07090086  0.48832858 ... -1.5210233   2.4034357
  -0.82028   ]
 [-1.2668697  -0.10823514  0.48824316 ... -1.5867457   2.5985293
  -0.8524832 ]
 ...
 [-1.1819224  -0.04187744  0.48389366 ... -1.5015293   2.4084098
  -0.8062623 ]
 [-1.2646203  -0.08763371  0.48651996 ... -1.5900754   2.670867
  -0.84927773]
 [-1.1805983  -0.03459521  0.48111358 ... -1.5008787   2.4264874
  -0.8051375 ]] 

y_pred is:
 [[0.02056659 0.0615592  0.1094998  ... 0.015137   0.60509545 0.02975604]
 [0.01797863 0.05590351 0.09779333 ... 0.0131117  0.6637885  0.02642334]
 [0.01485235 0.04731335 0.08590747 ... 0.01078636 0.70878845 0.02247817]
 ...
 [0.01849246 0.05782425 0.09782492 ... 0.01343356 0.67028004 0.02692415]
 [0.01425521 0.04625221 0.08212668 ... 0.01029509 0.72968274 0.02159507]
 [0.0183435  0.05770122 0.09663944 ... 0.01331638 0.6761135  0.02670193]] 



### Classification Report

In [28]:
y_pred_classes = np.argmax(y_pred, axis=1)
cr = classification_report(y_test2, y_pred_classes, target_names=[f'Cat {i+1}' for i in range(total_classes)])
print('Classification Report\n\n', cr)

Classification Report

               precision    recall  f1-score   support

       Cat 1       0.00      0.00      0.00        27
       Cat 2       0.00      0.00      0.00       116
       Cat 3       0.00      0.00      0.00       195
       Cat 4       0.00      0.00      0.00       320
       Cat 5       0.00      0.00      0.00        23
       Cat 6       0.66      1.00      0.79      1421
       Cat 7       0.00      0.00      0.00        52

    accuracy                           0.66      2154
   macro avg       0.09      0.14      0.11      2154
weighted avg       0.44      0.66      0.52      2154



c:\Users\chaud\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\chaud\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\chaud\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
